In [2]:
import os
from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
import requests
import urllib.request
import re
import time
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from IPython.display import display, HTML

In [3]:
yf.pdr_override()
pd.set_option('display.max_columns', None)
START_DATE = "2003-08-01"
END_DATE = "2015-01-01"

In [4]:
#Utility function to reformat yahoo finance numbers
def data_string_to_float(number_string):
    if ("N/A" in number_string) or ("NaN" in number_string):
        return "N/A"
    elif number_string == ">0":
        return 0
    elif "B" in number_string:
        return float(number_string.replace("B", "")) * 1000000000
    elif "M" in number_string:
        return float(number_string.replace("M", "")) * 1000000
    elif "K" in number_string:
        return float(number_string.replace("K", "")) * 1000
    else:
        return float(number_string)


In [5]:
#Get list of tickers and find price data
statspath = "C://Users//nickd//Downloads//intraQuarter//_KeyStats//"
ticker_list = os.listdir(statspath)
all_data = pdr.get_data_yahoo(ticker_list, START_DATE, END_DATE)
stock_data = all_data["Adj Close"]

[*********************100%***********************]  560 of 560 completed

141 Failed downloads:
- MJN: No data found for this date range, symbol may be delisted
- RAI: No data found for this date range, symbol may be delisted
- WAG: No data found for this date range, symbol may be delisted
- DTV: No data found, symbol may be delisted
- COH: Data doesn't exist for startDate = 1059714000, endDate = 1420092000
- CAB: Data doesn't exist for startDate = 1059714000, endDate = 1420092000
- LLTC: No data found for this date range, symbol may be delisted
- CVC: No data found for this date range, symbol may be delisted
- BKS: No data found, symbol may be delisted
- PCP: Data doesn't exist for startDate = 1059714000, endDate = 1420092000
- SCG: Data doesn't exist for startDate = 1059714000, endDate = 1420092000
- HCN: No data found for this date range, symbol may be delisted
- HOT: No data found for this date range, symbol may be delisted
- BBRY: No data found for this date range, symbol may be d

In [5]:
stock_data.dropna(how="all", axis=1, inplace=True)
missing_tickers = [
    ticker for ticker in ticker_list if ticker.upper() not in stock_data.columns
]
print(f"{len(missing_tickers)} tickers are missing: \n {missing_tickers} ")
stock_data.ffill(inplace=True)
stock_data.to_csv("stock_prices.csv")

C:\Users\nickd\AppData\Local\Temp\ipykernel_6696\4029096740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.dropna(how="all", axis=1, inplace=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_6696\4029096740.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.ffill(inplace=True)


141 tickers are missing: 
 ['ace', 'aci', 'act', 'adt', 'agn', 'altr', 'alxn', 'ann', 'apc', 'apol', 'arg', 'arna', 'aro', 'avp', 'bbry', 'bbt', 'bcr', 'beam', 'bhi', 'bks', 'bll', 'brcm', 'btu', 'cab', 'cam', 'cbg', 'cbs', 'cce', 'celg', 'cern', 'cfn', 'chk', 'cog', 'coh', 'ctl', 'cvc', 'dell', 'df', 'disca', 'dlph', 'dnb', 'dnr', 'do', 'dow', 'dps', 'dsw', 'dtv', 'esv', 'etfc', 'fb', 'fdo', 'flir', 'frx', 'ftr', 'gmcr', 'hcbk', 'hcn', 'hcp', 'hot', 'hrs', 'hsp', 'htz', 'ir', 'jcp', 'jdsu', 'jec', 'josb', 'joy', 'krft', 'ksu', 'life', 'lltc', 'lm', 'lo', 'ltd', 'luk', 'mhfi', 'mjn', 'mon', 'mwv', 'myl', 'nbl', 'ne', 'ntri', 'nu', 'pbct', 'pcl', 'pcln', 'pcp', 'petm', 'pll', 'pvtb', 'px', 'qep', 'rai', 'rdc', 'rht', 'rtn', 's', 'sai', 'scg', 'sd', 'se', 'sfly', 'shld', 'sial', 'sndk', 'sne', 'spls', 'sti', 'stj', 'swy', 'symc', 'te', 'teg', 'tibx', 'tif', 'tmk', 'tso', 'tss', 'twc', 'tyc', 'ua', 'utx', 'var', 'viab', 'vitc', 'vprt', 'wag', 'wfm', 'win', 'wlp', 'wpo', 'wpx', 'wyn', 'xl'

In [6]:
index_data = pdr.get_data_yahoo("SPY", start=START_DATE, end=END_DATE)
index_data.to_csv("sp500_index.csv")

[*********************100%***********************]  1 of 1 completed


In [7]:
df = pd.DataFrame()

for ticker in ticker_list:
    ticker = ticker.upper()

    stock_ohlc = pdr.get_data_yahoo(ticker, START_DATE, END_DATE)
    if stock_ohlc.empty:
        print(f"No data for {ticker}")
        continue
    adj_close = stock_ohlc["Adj Close"].rename(ticker)
    df = pd.concat([df, adj_close], axis=1)
df.to_csv("stock_prices.csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACE: Data doesn't exist for startDate = 1059714000, endDate = 1420092000
No data for ACE
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACI: Data doesn't exist for startDate = 1059714000, endDate = 1420092000
No data for ACI
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACT: Data doesn't exist for startDate = 1059714000, endDate = 1420

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ESV: No data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NTRI: No data found, symbol may be delisted
No data for NTRI
[*******************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SFLY: No data found, symbol may be delisted
No data for SFLY
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SHLD: Data doesn't exist for startDate = 1059714000, endDate = 1420092000
No data for SHLD
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SIAL: No data found for this date range, symbol may be delisted
No data for SIAL
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**********

In [6]:
features = ["Total Debt/Equity",
                      'Trailing P/E',
                      'Price/Book',
                      'Profit Margin',
                      'Operating Margin',
                      'Return on Assets',
                      'Return on Equity',
                      'Revenue Per Share',
                      'Market Cap',
                        'Enterprise Value',
                        'Forward P/E',
                        'PEG Ratio',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Revenue',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common ',
                        'Diluted EPS',
                        'Qtrly Earnings Growth',
                        'Qtrly Revenue Growth',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Cash Flow',
                        'Beta',
                        '% Held by Insiders',
                        '% Held by Institutions',
                        'Shares Short (as of',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short (prior ',
                        'Current Price',
                        'Shares Outstanding' ]


In [7]:
# Read in SP500 data and stock data, parsing the dates.
sp500_raw_data = pd.read_csv("sp500_index.csv", index_col="Date", parse_dates=True)
stock_raw_data = pd.read_csv("stock_prices.csv", index_col="Date", parse_dates=True)

In [8]:
# We will reindex to include the weekends.
start_date = str(stock_raw_data.index[0])
end_date = str(stock_raw_data.index[-1])
idx = pd.date_range(start_date, end_date)
sp500_raw_data = sp500_raw_data.reindex(idx)
stock_raw_data = stock_raw_data.reindex(idx)

In [9]:
# Now the weekends are NaN, so we fill forward these NaNs
# (i.e weekends take the value of Friday's adjusted close).
sp500_raw_data.ffill(inplace=True)
stock_raw_data.ffill(inplace=True)
# The tickers whose data is to be parsed.
stock_list = [x[0] for x in os.walk(statspath)]
stock_list = stock_list[1:]

In [10]:
 # Creating a new dataframe which we will later fill.
df_columns = [
    "Date",
    "Unix",
    "Ticker",
    "Price",
    "stock_p_change",
    "SP500",
    "SP500_p_change",
] + features

df = pd.DataFrame(columns=df_columns)

In [23]:
# tqdm is a simple progress bar
for stock_directory in tqdm(stock_list, desc="Parsing progress:", unit="tickers"):
    keystats_html_files = os.listdir(stock_directory)

    ticker = stock_directory.split(statspath)[1]

    for file in keystats_html_files:
            # Convert the datetime format of our file to unix time
        date_stamp = datetime.strptime(file, "%Y%m%d%H%M%S.html")
        unix_time = time.mktime(date_stamp.timetuple())

            # Read in the html file as a string.
        full_file_path = stock_directory + "/" + file

            # This will store the parsed values
        value_list = []

        with open(full_file_path, "r") as source:
            source = source.read()
                # Remove commas from the html to make parsing easier.
            source = source.replace(",", "")

                # Regex search for the different variables in the html file, then append to value_list
            for variable in features:
                if variable == 'Current Price':
                    try:
                        value_list.append(0)
                    except KeyError:
                        continue
                else:
                
                    
                    # Search for the table entry adjacent to the variable name.
                    try:
                        regex = (
                            r">"
                            + re.escape(variable)
                             + r".*?(\-?\d+\.*\d*K?M?B?|N/A[\\n|\s]*|>0|NaN)%?"
                             r"(</td>|</span>)"
                        )
                        value = re.search(regex, source, flags=re.DOTALL).group(1)

                        # Dealing with number formatting
                        value_list.append(data_string_to_float(value))

                    # The data may not be present. Process accordingly
                    except AttributeError:
                        # In the past, 'Avg Vol' was instead named 'Average Volume'
                        # If 'Avg Vol' fails, search for 'Average Volume'.
                        if variable == "Avg Vol (3 month)":
                            try:
                                new_variable = ">Average Volume (3 month)"
                                regex = (
                                    re.escape(new_variable)
                                    + r".*?(\-?\d+\.*\d*K?M?B?|N/A[\\n|\s]*|>0)%?"
                                    r"(</td>|</span>)"
                                )
                                value = re.search(regex, source, flags=re.DOTALL).group(
                                       1
                                )
                                value_list.append(data_string_to_float(value))
                            except AttributeError:
                                value_list.append("N/A")
                        else:
                            value_list.append("N/A")

            current_date = datetime.fromtimestamp(unix_time).strftime("%Y-%m-%d")
            one_year_later = datetime.fromtimestamp(unix_time + 31536000).strftime(
                 "%Y-%m-%d"
            )

            # SP500 prices now and one year later, and the percentage change
            sp500_price = float(sp500_raw_data.loc[current_date, "Adj Close"])
            sp500_1y_price = float(sp500_raw_data.loc[one_year_later, "Adj Close"])
            sp500_p_change = round(
                ((sp500_1y_price - sp500_price) / sp500_price * 100), 2
            )

            # Stock prices now and one year later. We need a try/except because some data is missing
            stock_price, stock_1y_price = "N/A", "N/A"
            try:
                stock_price = float(stock_raw_data.loc[current_date, ticker.upper()])
                stock_1y_price = float(stock_raw_data.loc[one_year_later, ticker.upper()])
            except KeyError:
                # If stock data is missing, we must skip this datapoint
                print(f"PRICE RETRIEVAL ERROR for {ticker}")
                continue

            stock_p_change = round(
                ((stock_1y_price - stock_price) / stock_price * 100), 2
            )
        # Append all our data to the dataframe.
            new_df_row = [
                date_stamp,
                unix_time,
                ticker,
                stock_price,
                stock_p_change,
                sp500_price,
                sp500_p_change,
            ] + value_list
            df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
    # Remove rows with missing stock price data
    df.dropna(axis=0, subset=["Price", "stock_p_change"], inplace=True)
    # Output the CSV
    df.to_csv("keystats.csv", index=False)

Parsing progress::   0%|                                                                  | 0/560 [00:00<?, ?tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:10

C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict(zip(df_columns, new_df_row)), ignore_index=True)
C:\Users\nickd\AppData\Local\Temp\ipykernel_5084\3625399479.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace
PRICE RETRIEVAL ERROR for ace


...

Parsing progress:: 100%|███████████████████████████████████████████████████████▊| 558/560 [07:35<00:01,  1.12tickers/s]

PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh
PRICE RETRIEVAL ERROR for zmh


Parsing progress:: 100%|███████████████████████████████████████████████████████▉| 559/560 [07:36<00:00,  1.25tickers/s]

PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga
PRICE RETRIEVAL ERROR for znga


Parsing progress:: 100%|████████████████████████████████████████████████████████| 560/560 [07:37<00:00,  1.23tickers/s]


In [11]:
def status_calc(stock, sp500, outperformance=10):
    """A simple function to classify whether a stock outperformed the S&P500
    :param stock: stock price
    :param sp500: S&P500 price
    :param outperformance: stock is classified 1 if stock price > S&P500 price + outperformance
    :return: true/false
    """
    if outperformance < 0:
        raise ValueError("outperformance must be positive")
    return stock - sp500 >= outperformance

In [26]:
def Build_Data_Set():
    data_df = pd.read_csv("keystats.csv")
    data_df.drop_duplicates()
    data_df.dropna(inplace=True)
    data_df = data_df.reindex(np.random.permutation(data_df.index))
    features = data_df.columns[4:]
    X = np.array(data_df[features].values)
    X = preprocessing.scale(X)

    y = list(
        status_calc(
            data_df["stock_p_change"], data_df["SP500_p_change"], outperformance=10
        )
    )


    Z = np.array(data_df[["stock_p_change","SP500_p_change"]])


    return X,y,Z

test_size = 1000

invest_amount = 10000
total_invests = 0
if_market = 0
if_strat = 0



    
X, y, Z = Build_Data_Set()
print(len(X))

    
clf = RandomForestClassifier(n_estimators=100, random_state=1)
clf.fit(X[:-test_size],y[:-test_size])

correct_count = 0

for x in range(1, test_size+1):
    if clf.predict([X[-x]])[0] == y[-x]:
        correct_count += 1

    if clf.predict([X[-x]])[0] == 1:
        invest_return = invest_amount + (invest_amount * (Z[-x][0]/100))
        market_return = invest_amount + (invest_amount * (Z[-x][1]/100))
        total_invests += 1
        if_market += market_return
        if_strat += invest_return
            

print("Accuracy:", (correct_count/test_size) * 100.00)

print("Total Trades:", total_invests)
print("Ending with Strategy:",if_strat)
print("Ending with Market:",if_market)

compared = ((if_strat - if_market) / if_market) * 100.0
do_nothing = total_invests * invest_amount

avg_market = ((if_market - do_nothing) / do_nothing) * 100.0
avg_strat = ((if_strat - do_nothing) / do_nothing) * 100.0


    
print("Compared to market, we earn",str(compared)+"% more")
print("Average investment return:", str(avg_strat)+"%")
print("Average market return:", str(avg_market)+"%")

C:\Users\nickd\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(


[False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 Fal

3149
Accuracy: 90.2
Total Trades: 297
Ending with Strategy: 4414467.0
Ending with Market: 3389720.0
Compared to market, we earn 30.231022031318222% more
Average investment return: 48.635252525252525%
Average market return: 14.131986531986533%


In [42]:
data_df = pd.read_csv("keystats.csv")
data_df.drop_duplicates()
data_df.dropna(inplace=True)
data_df = data_df.reindex(np.random.permutation(data_df.index))
features = data_df.columns[4:]
X = np.array(data_df[features].values)
sc = MinMaxScaler(feature_range=(0,1))
X = sc.fit_transform(X)
y = np.array(
    status_calc(
        data_df["stock_p_change"], data_df["SP500_p_change"], outperformance=10
    )
)
y.astype(int)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X,y,epochs=100,batch_size=32)


Epoch 1/100


KeyboardInterrupt: 

In [12]:
if not os.path.exists("forward_json/"):
    os.makedirs("forward_json/")

    # Retrieve a list of tickers from the fundamental data folder
ticker_list = os.listdir(statspath)

In [36]:
for ticker in tqdm(ticker_list, desc="Download progress:", unit="tickers"):
        try:
            link = f"https://query2.finance.yahoo.com/v10/finance/quoteSummary/"+ticker.upper()+"?modules=assetProfile,financialData,defaultKeyStatistics,calendarEvents,incomeStatementHistory,cashflowStatementHistory,balanceSheetHistory"
            resp = urllib.request.urlopen(link).read()

            # Write results to forward/
            save = f"forward_json/"+str(ticker)+".json"
            store = open(save,"w")
            store.write(str(resp))
            store.close()
        except Exception as e:
            print(f"{ticker}: {str(e)}\n")
            time.sleep(2)

Download progress::   1%|▌                                                        | 6/560 [00:01<02:24,  3.84tickers/s]

ace: HTTP Error 404: Not Found



Download progress::   4%|██                                                      | 20/560 [00:08<03:05,  2.91tickers/s]

aet: HTTP Error 404: Not Found



Download progress::   4%|██▏                                                     | 22/560 [00:10<06:54,  1.30tickers/s]

...

Download progress:: 100%|██████████████████████████████████████████████████████▊| 558/560 [06:58<00:02,  1.14s/tickers]

zmh: HTTP Error 404: Not Found



Download progress:: 100%|██████████████████████████████████████████████████████▉| 559/560 [07:01<00:01,  1.44s/tickers]

znga: HTTP Error 404: Not Found



Download progress:: 100%|███████████████████████████████████████████████████████| 560/560 [07:03<00:00,  1.32tickers/s]


In [13]:
gather=['debtToEquity',
            'Trailing P/E', ## Added a custom calculation, look down
            'priceToBook',
            'profitMargins',
            'operatingMargins',
            'returnOnAssets',
            'returnOnEquity',
            'revenuePerShare',
            'Market Cap', ## Leaving this here to avoid changing all the numbering for the list
            'enterpriseValue',
            'forwardPE',
            'pegRatio',
            'enterpriseToRevenue',
            'enterpriseToEbitda',
            'totalRevenue',
            'grossProfit',
            'ebitda',
            'netIncomeToCommon',
            'trailingEps',
            'earningsGrowth',
            'revenueGrowth',
            'totalCash',
            'totalCashPerShare',
            'totalDebt',
            'currentRatio',
            'bookValue',
            'operatingCashflow',
            'beta',
            'heldPercentInsiders',
            'heldPercentInstitutions',
            'sharesShort',
            'shortRatio',
            'shortPercentOfFloat',
            'sharesShortPriorMonth',
			'currentPrice',
			'sharesOutstanding']

In [14]:
df = pd.DataFrame(columns = ['Date',
'Unix',
'Ticker',
'Price',
'stock_p_change',
'SP500',
'SP500_p_change',
##############
'DE Ratio',
'Trailing P/E',
'Price/Book',
'Profit Margin',
'Operating Margin',
'Return on Assets',
'Return on Equity',
'Revenue Per Share',
'Market Cap',
'Enterprise Value',
'Forward P/E',
'PEG Ratio',
'Enterprise Value/Revenue',
'Enterprise Value/EBITDA',
'Revenue',
'Gross Profit',
'EBITDA',
'Net Income Avl to Common ',
'Diluted EPS',
'Earnings Growth',
'Revenue Growth',
'Total Cash',
'Total Cash Per Share',
'Total Debt',
'Current Ratio',
'Book Value Per Share',
'Cash Flow',
'Beta',
'Held by Insiders',
'Held by Institutions',
'Shares Short (as of',
'Short Ratio',
'Short % of Float',
'Shares Short (prior ',
'Current Price',
'Shares Outstanding',                              
])

In [52]:
tickerfile_list = os.listdir("forward_json/")

In [53]:
 for tickerfile in tqdm(tickerfile_list, desc="Parsing progress:", unit="tickers"):
        ticker = tickerfile.split(".json")[0].upper()
        source = open(f"forward_json/{tickerfile}").read()
        # Remove commas from the html to make parsing easier.
        source = source.replace(",", "")

        # Regex search for the different variables in the html file, then append to value_list
        value_list = []
        for variable in gather:
            try:    

                regex = re.escape(variable) + r'.*?"(-?\d{1,8}\.\d{1,8}M?B?K?|N/A)%?'
                value = re.search(regex, source)
                value = value.group(1)

                if "B" in value:
                    value = float(value.replace("B",'')) * 1000000000

                elif "M" in value:
                    value = float(value.replace("M",'')) * 1000000

                elif "K" in value:
                    value = float(value.replace("K",'')) * 1000

                value_list.append(value)

            except Exception as e:
                value = "N/A"
                value_list.append(value)

        if value_list.count("N/A") > 15:
		        pass

        else:

            df = df.append({'Date':0,
		                        'Unix':0,
		                        'Ticker':ticker, ## Getting Only The Stock Name, not 'json'
		                        'Price':0,
		                        'stock_p_change':0,
		                        'SP500':0,
                                'SP500_p_change':0,
		                        'DE Ratio':value_list[0],
		                        'Trailing P/E':float(value_list[34]) / float(value_list[18]) ,
		                        'Price/Book':value_list[2],
		                        'Profit Margin':value_list[3],
		                        'Operating Margin':value_list[4],
		                        'Return on Assets':value_list[5],
		                        'Return on Equity':value_list[6],
		                        'Revenue Per Share':value_list[7],
		                        'Market Cap': float(value_list[35]) * float(value_list[34]) , #Multiplying Shares Outstanding * Current Price to determine Market Cap
		                        'Enterprise Value':value_list[9],
		                        'Forward P/E':value_list[10],
		                        'PEG Ratio':value_list[11],
		                        'Enterprise Value/Revenue':value_list[12],
		                        'Enterprise Value/EBITDA':value_list[13],
		                        'Revenue':value_list[14],
		                        'Gross Profit':value_list[15],
		                        'EBITDA':value_list[16],
		                        'Net Income Avl to Common ':value_list[17],
		                        'Diluted EPS':value_list[18],
		                        'Earnings Growth':value_list[19],
		                        'Revenue Growth':value_list[20],
		                        'Total Cash':value_list[21],
		                        'Total Cash Per Share':value_list[22],
		                        'Total Debt':value_list[23],
		                        'Current Ratio':value_list[24],
		                        'Book Value Per Share':value_list[25],
		                        'Cash Flow':value_list[26],
		                        'Beta':value_list[27],
		                        'Held by Insiders':value_list[28],
		                        'Held by Institutions':value_list[29],
		                        'Shares Short (as of':value_list[30],
		                        'Short Ratio':value_list[31],
		                        'Short % of Float':value_list[32],
		                        'Shares Short (prior ':value_list[33],
                                'Current Price': value_list[34],
                                'Shares Outstanding': value_list[35]}, ignore_index = True)
df.to_csv("forward_sample.csv", index=False)

Parsing progress::   0%|                                                                  | 0/423 [00:00<?, ?tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Parsing progress::  10%|█████▌                                                   | 41/423 [00:00<00:03, 96.81tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
Parsing progress::  20%|███████████▎                                             | 84/423 [00:00<00:03, 96.24tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Parsing progress::  29%|████████████████▍                                       | 124/423 [00:01<00:03, 92.29tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Parsing progress::  39%|█████████████████████▋                                  | 164/423 [00:01<00:02, 90.63tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

...

C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
Parsing progress::  58%|████████████████████████████████▎                       | 244/423 [00:02<00:02, 88.27tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
Parsing progress::  66%|█████████████████████████████████████                   | 280/423 [00:03<00:01, 88.06tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Parsing progress::  76%|██████████████████████████████████████████▎             | 320/423 [00:03<00:01, 94.72tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Parsing progress::  85%|███████████████████████████████████████████████▋        | 360/423 [00:03<00:00, 94.89tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

Parsing progress::  95%|████████████████████████████████████████████████████▉   | 400/423 [00:04<00:00, 89.88tickers/s]C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date':0,
C:\Users\nickd\AppData\Local\Temp\ipykernel_6404\359506910.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [45]:
# The percentage by which a stock has to beat the S&P500 to be considered a 'buy'
OUTPERFORMANCE = 10


def build_data_set():
    """
    Reads the keystats.csv file and prepares it for scikit-learn
    :return: X_train and y_train numpy arrays
    """
    data_df = pd.read_csv("keystats.csv")
    data_df.drop_duplicates()
    data_df.dropna(inplace=True)
    data_df = data_df.reindex(np.random.permutation(data_df.index))
    features = data_df.columns[7:]
    X = np.array(data_df[features].values)
    X_train = data_df[features].values
    # Generate the labels: '1' if a stock beats the S&P500 by more than 10%, else '0'.
    y_train = list(
        status_calc(
            data_df["stock_p_change"],
            data_df["SP500_p_change"],
            OUTPERFORMANCE,
        )
    )

    return X_train, y_train

In [46]:
X_train, y_train = build_data_set()

clf = RandomForestClassifier(n_estimators=100, random_state=0, criterion = "log_loss")
clf.fit(X_train, y_train)

    # Now we get the actual data from which we want to generate predictions.
data = pd.read_csv("forward_sample.csv")
data.dropna(inplace=True)
data.drop_duplicates()
data= data.reindex(np.random.permutation(data.index))
features = data.columns[7:]
X_test = data[features].values
z = data["Ticker"].values

    # Get the predicted tickers
y_pred = clf.predict(X_test)
if sum(y_pred) == 0:
    print("No stocks predicted!")
else:
    invest_list = z[y_pred].tolist()
    pList = [*set(invest_list)]
    print(
        f"{len(pList)} stocks predicted to outperform the S&P500 by more than {OUTPERFORMANCE}%:"
    )
    print(" ".join(pList))

96 stocks predicted to outperform the S&P500 by more than 10%:
WTW GNW S GME MAS TRIP PH LIFE RL NBR DKS HTZ ANF HUM SCHL AIZ ADT CAH FLWS PETS PLL MAC SEE HCP CMI PX NRG MON CLDX LNC X BTU AMP SWN UA CLF BIG EXPR WYNN UNM NWSA XL NOV PHM BLK KSS BWA RRC MSI R AIV SRCL DLX APA DO SLM URBN FTI WHR FL CNX DDS CIM BBBY APD ZION TDC NE CAMP FOSL STX CRM SD LRCX AN SE BEAM PBI GPS LLL LH AA GRPN GCI XRX ROP M NILE WGO SAI CCL ALTR EMN YUM HOV WU


In [39]:
OUTPERFORMANCE = 10
data_df = pd.read_csv("keystats.csv")
data_df.drop_duplicates()
data_df.dropna(inplace=True)
data_df = data_df.reindex(np.random.permutation(data_df.index))
features = data_df.columns[7:]
X = np.array(data_df[features].values)
sc = MinMaxScaler(feature_range=(0,1))
X = sc.fit_transform(X)
y_train = np.array(
    status_calc(
        data_df["stock_p_change"],
        data_df["SP500_p_change"],
        OUTPERFORMANCE,
    )
)
y_train = y_train.astype(int)
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X,y_train,epochs=100,batch_size=32)
y_pred = model.predict(X)
display(y_pred)

Epoch 1/100
99/99 [==============================] - 8s 38ms/step - loss: 0.2304
Epoch 2/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2244
Epoch 3/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2237
Epoch 4/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2216
Epoch 5/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2214
Epoch 6/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2222
Epoch 7/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2203
Epoch 8/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2214
Epoch 9/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2216
Epoch 10/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2207
Epoch 11/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2196
Epoch 12/100
99/99 [==============================] - 4s 38ms/step - loss: 0.2200
Epoch 13/100
99/99 [=====

99/99 [==============================] - 2s 15ms/step


array([[0.46762937],
       [0.3362614 ],
       [0.3940581 ],
       ...,
       [0.28929663],
       [0.28602436],
       [0.4539696 ]], dtype=float32)